In [1]:
# Naver News API Crawling
import requests
import pprint
from CommonFile import remove_html_tags_regex, data_type_check, create_excel_file

# 현재 작업 디렉토리의 상위 폴더를 경로에 추가
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# Naver API Set Up
CLIENT_ID = Personal_CLIENT_ID
CLIENT_SECRET = Personal_CLIENT_SECRET

# Naver API Crawling Function
# def naver_api_crawling(query='샤오미', pages=2):
def naver_api_crawling(query, pages, CLIENT_ID, CLIENT_SECRET):
    
    results = []

    for page in range(pages):

        try:            
            start = 1 + (page * 100)
            # start = 1 + page
            
            url = f'https://openapi.naver.com/v1/search/news.json?query={query}&display=100&start={start}'
            headers = {
                "X-Naver-Client-Id": CLIENT_ID,
                "X-Naver-Client-Secret": CLIENT_SECRET
            }

            print(url)
            
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # HTTP 에러 체크

            # call json api
            data = response.json()
            # pprint.pprint(data.get('items', []))

            for index, item in enumerate(data.get('items', [])):
                # pprint.pprint(item)
                
                # remove tag into title
                title = remove_html_tags_regex(item['title'])
                link = item['link']

                # data_type_check - change Data Type
                pubDate = data_type_check(item['pubDate'])

                # 받아온 리스트 확인
                # print(index + 1, title, link, pubDate)
                
                # page=0일 때: offset=0   (1페이지 → 1~100위)
                # page=1일 때: offset=100 (2페이지 → 101~200위)  
                # page=2일 때: offset=200 (3페이지 → 201~300위)
                actual_rank = start + index 
                results.append([actual_rank, title, link, pubDate])
                # print(results)
            
        # raise_for_status()가 발생시킨 HTTPError를 잡음
        except requests.exceptions.HTTPError as e:
            # print(f"HTTPError 에러: {e}")
            print(f"페이지 {page+1} HTTPError: {e}")
            continue  # 다음 페이지 계속 진행
        except requests.RequestException as e: # 네트워크 에러[전체 포괄]
            print(f"네트워크 에러: {e}")   
        except ValueError as e:
            print(f"JSON 파싱 에러: {e}")
        except Exception as e:            
            print(f"알 수 없는 에러: {e}")
            
    return results        


# Function execution
# [다른 페이지에서 호출시 페이지 로드만으로 실행되지 않고 함수 호출시 실행]
if __name__ == "__main__":
    
    # print('실행')
    news_data = naver_api_crawling('지하철', 2, CLIENT_ID, CLIENT_SECRET)
    create_excel_file('Naver-News-Data', '네이버 지하철 관련 뉴스', news_data, "Data")


https://openapi.naver.com/v1/search/news.json?query=지하철&display=100&start=1
https://openapi.naver.com/v1/search/news.json?query=지하철&display=100&start=101
Data\Naver-News-Data.xlsx
저장 완료: Data\Naver-News-Data.xlsx (200개 항목)
